# AI Skills + OpenAI Quick Start

This notebook demonstrates how to use AI Skills with OpenAI models (GPT-4, GPT-3.5, etc.).

## Setup

First, install the required packages:

In [ ]:
!pip install aiskills[openai,search] -q

## Configure API Key

Set your OpenAI API key:

In [ ]:
import os

# Option 1: Set directly (not recommended for production)
# os.environ["OPENAI_API_KEY"] = "sk-your-key-here"

# Option 2: Use getpass for secure input
from getpass import getpass
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

## Quick Start: 3 Lines of Code

The simplest way to use AI Skills with OpenAI:

In [ ]:
from aiskills.integrations import create_openai_client

client = create_openai_client(model="gpt-4")
response = client.chat("Help me debug a memory leak in Python")
print(response)

## How It Works

The client automatically:
1. Sends your message to GPT-4
2. If GPT-4 decides to use a skill, it makes a tool call
3. The client executes the tool call (searches/loads skills)
4. Results are sent back to GPT-4
5. GPT-4 generates the final response using the skill content

## Available Skills

Let's see what skills are available:

In [ ]:
# List available skills
skills = client.list_skills()
print(f"Found {len(skills)} skills:\n")
for skill in skills[:10]:  # Show first 10
    print(f"  - {skill['name']}: {skill['description'][:60]}...")

## Search for Skills

You can search for specific skills:

In [ ]:
# Search for testing-related skills
results = client.search_skills("unit testing python")
print(f"Found {results.total} skills for 'unit testing python':\n")
for r in results.results:
    score = r.get('score', 'N/A')
    print(f"  - {r['name']} (score: {score})")

## Read a Specific Skill

Load a skill by name:

In [ ]:
# Read a specific skill
result = client.read_skill("testing-strategies")  # Change to a skill that exists
if result.content:
    print(f"Skill: {result.skill_name}")
    print(f"Content preview:\n{result.content[:500]}...")
else:
    print(f"Error: {result.error}")

## Multi-Turn Conversation

For conversations, use `chat_with_messages`:

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful coding assistant with access to AI skills."},
    {"role": "user", "content": "What's the best way to handle errors in Python?"}
]

response = client.chat_with_messages(messages)
print(response)

## Manual Tool Handling

For more control, disable auto-execution:

In [ ]:
from aiskills.integrations import create_openai_client
import json

# Create client without auto-execution
manual_client = create_openai_client(auto_execute=False)

# Get raw response
response = manual_client.get_completion_with_skills(
    messages=[{"role": "user", "content": "How do I optimize SQL queries?"}]
)

# Check for tool calls
if response.choices[0].message.tool_calls:
    for call in response.choices[0].message.tool_calls:
        print(f"Tool call: {call.function.name}")
        print(f"Arguments: {call.function.arguments}")
        
        # Execute manually
        result = manual_client.execute_tool(
            call.function.name,
            json.loads(call.function.arguments)
        )
        print(f"Result: {str(result)[:200]}...")
else:
    print("No tool calls made")
    print(response.choices[0].message.content)

## Using with Different Models

You can use any OpenAI model:

In [ ]:
# GPT-3.5 Turbo (faster, cheaper)
fast_client = create_openai_client(model="gpt-3.5-turbo")
response = fast_client.chat("Quick question: how do I reverse a string in Python?")
print(response)

## Next Steps

- Install more skills: `aiskills install <source>`
- Create your own skills: `aiskills init my-skill`
- Run the API server: `aiskills api serve`

For more information, see the [documentation](https://github.com/sergioc/ai-skills).